## Lab 7: Eel Distribution Modeling with XGBoost

**Reference Paper:** [Elith et al. (2008)](https://ucsb.box.com/s/6k7636wsbogdg3orarxrlowke0ounbic)

In this lab, you will model the distribution of the eel species *Anguilla australis* using **boosted classification trees (BCTs)**, a machine learning technique that improves predictive performance by combining multiple decision trees. Elith et al. (2008) offered an early implementation of BRTs in an ecological setting to understand how environmental variables influence eel distribution.

You will work with **two datasets**:
1. **Training Data** – Used to build and evaluate your XGBoost model.
2. **Evaluation Data** – Used to assess model performance on unseen data.

To achieve the following objectives:
- Train and fine-tune an **XGBoost** model for classification of species presence/absence data.
- Compare your model’s performance to the approach used by Elith et al.


**Wherever applicable in this lab, use a random state of 808.**

### Step 0: Load libraries and data


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split,RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from scipy.stats import uniform, randint

# Download the datasets
model_data = pd.read_csv("/courses/EDS232/Data/model.data.csv").drop(columns=['Site'])
eval_data = pd.read_csv("/courses/EDS232/Data/eval.data.csv")

### Step 1:Initial Data Preprocessing
Let's get started by preparing our data. `Angaus` will be our target variable(`y`), and all other variables will be our features (`X`). Then encode the categorical feature using `LabelEncoder()`. The final step will be a bit different this time.  We don't need to split off testing data for the final model evaluation; a separate set (`eval_data`) will be used as in Elith et al.  We do, however, need to split our data in order to do the early stopping process. When splitting your data into training and validation, use a test size of 0.2 and a random state of 808. 

In [2]:
# Look at model data
model_data.head()

,Angaus,SegSumT,SegTSeas,SegLowFlow,DSDist,DSMaxSlope,USAvgT,USRainDays,USSlope,USNative,DSDam,Method,LocSed
0,0,16.0,-0.10,1.036,50.20,0.57,0.09,2.470,9.8,0.81,0,electric,4.8
1,1,18.7,1.51,1.003,132.53,1.15,0.20,1.153,8.3,0.34,0,electric,2.0
2,0,18.3,0.37,1.001,107.44,0.57,0.49,0.847,0.4,0.00,0,spo,1.0
3,0,16.7,-3.80,1.000,166.82,1.72,0.90,0.210,0.4,0.22,1,electric,4.0
4,1,17.2,0.33,1.005,3.95,1.15,-1.20,1.980,21.9,0.96,0,electric,4.7


In [3]:
# Select features
X = model_data.drop('Angaus', axis = 1).copy()
# Select target
y = model_data['Angaus']

In [4]:
# Recode categorical predictors to integers
le = LabelEncoder()
categorical = X.select_dtypes(include = ['object'])

for column in categorical:
    X[column] = le.fit_transform(X[column])

# Check
X.head()

,SegSumT,SegTSeas,SegLowFlow,DSDist,DSMaxSlope,USAvgT,USRainDays,USSlope,USNative,DSDam,Method,LocSed
0,16.0,-0.10,1.036,50.20,0.57,0.09,2.470,9.8,0.81,0,0,4.8
1,18.7,1.51,1.003,132.53,1.15,0.20,1.153,8.3,0.34,0,0,2.0
2,18.3,0.37,1.001,107.44,0.57,0.49,0.847,0.4,0.00,0,3,1.0
3,16.7,-3.80,1.000,166.82,1.72,0.90,0.210,0.4,0.22,1,0,4.0
4,17.2,0.33,1.005,3.95,1.15,-1.20,1.980,21.9,0.96,0,0,4.7


In [5]:
# Split the model data into testing and validation sets
X_train, X_val, y_train, y_val = train_test_split(X,y, test_size = 0.2, random_state = 808)

### Step 2: Determine best number of trees using early stopping
As a guard against overfitting while maximizing performance, we use **early stopping**. We start with a large number of trees and allow XGBoost to determine the optimal number by stopping training when the validation error no longer improves.

The choice of hyperparameter starting values is important in this process. We begin with:
- `n_estimators=1000` to ensure the model has enough capacity to learn meaningful patterns.
- `learning_rate=0.1` as a reasonable default that balances learning speed and performance.
- `eval_metric="logloss"` as the metric of performance to optimize.
- `early_stopping_rounds=50` to halt training if no improvement is seen for 50 rounds, preventing unnecessary computations.
- `random_state = 808`

We then `fit()` our specified baseline model, passing in the training sets as usual and specifying validation sets values for the `eval_set` parameter.

Finally, get and print the best number of trees from the fitted baseline model.


In [6]:
# Define initial XGBoost model
model = xgb.XGBClassifier(n_estimators = 1000, 
                          learning_rate = 0.1, 
                          eval_metric = "logloss", 
                          early_stopping_rounds = 50, 
                          random_state = 808,
                         n_jobs = -1)

# Fit initial model
model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose = 0)

print(f"Best number of trees (n_estimators): {model.best_iteration}")

Best number of trees (n_estimators): 29


### Step 3: Tune Learning Rate

The (`learning_rate` hyperparameter controls how much each tree contributes to improving the model's performance. A *higher* learning rate allows the model to learn quickly but risks missing the optimal solution and overfitting, while a *lower* learning rate makes learning slower but can improve generalization.

To find the optimal value, we'll use **randomized search cross-validation** (`RandomizedSearchCV`) to test different learning rates in the 0.01 to 0.3 range. Instead of testing every possible value, this method samples a set number of candidates (`n_iter`) from a defined parameter distribution.  In this case, sampling 20 candidates from a uniform distribution between `0.01` and `0.31`. Check out the [documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.uniform.html) on `scipy.stats.uniform` to see how it differs from `random.uniform`. Be sure to use a random state of 808.

After using `RandomizedSearchCV`, fit your model. Print the best learning rate. 


In [7]:
# Define initial XGBoost model
model = xgb.XGBClassifier(n_estimators = 29,  
                          random_state = 808,
                         n_jobs = -1)

In [8]:
# Define parameters
param_dist = {
    "learning_rate": uniform(0.01, 0.3)
}

# Set up RandomizedSearchCV
random_search = RandomizedSearchCV(
    model, 
    param_distributions = param_dist,  
    n_iter = 20, 
    verbose = 1, 
    random_state = 808)

# Run random search
random_search.fit(X_train, y_train, verbose = 0)

print(f'The best learning rate is: {random_search.best_params_["learning_rate"]}')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
The best learning rate is: 0.04204913369557677


### Step 4: Tune Tree-Specific Parameters

Now that we've determined the best number of tree and learning rate, we need to tune the complexity of individual trees in our model. Initialize your model with the best number of trees and learning rate.Then, define a parameter dictionary that takes on the following values:  

- `max_depth`(Controls how deep each tree can grow.  Takes integer values): A random integer from 3 to 10 ( inclusive of 3 and 10)
- `min_child_weight`( Determines the minimum number of samples required in a leaf node. Takes integer values) : A random integer from 1 to 10 ( inclusive of 1 and 10)
- `gamma` (Defines the minimum loss reduction needed to make a further split in a tree. Can take on values from a continuous range):  A uniform distribution from 0.05 to 0.10 - once again remember to check the `scipy.stats.uniform()` documentation! 
- `random_state = 808`

To find the best combination, we again use `RandomizedSearchCV`, allowing us to efficiently sample hyperparameters and evaluate different configurations using cross-validation. After fitting the model, print the best parameters. 



In [9]:
# Define subsequent model
model_2 = xgb.XGBClassifier(n_estimators = 29, 
                          learning_rate = random_search.best_params_["learning_rate"],
                            n_jobs = -1,
                          random_state = 808)

# Define tree-specific parameters
param_dist = {
    "max_depth": randint(3, 11),
    "min_child_weight": randint(1, 11),
    "gamma": uniform(0.05, 0.05),
}

# Set up RandomizedSearchCV
random_search_2 = RandomizedSearchCV(
    model_2, 
    param_distributions = param_dist, 
    n_iter = 20, 
    verbose = 0, 
    random_state = 808)

# Run random search
random_search_2.fit(X_train, y_train, verbose = 0)

print(f'The best max depth is: {random_search_2.best_params_["max_depth"]}')
print(f'The best min child weight is: {random_search_2.best_params_["min_child_weight"]}')
print(f'The best gamma is: {random_search_2.best_params_["gamma"]}')

The best max depth is: 9
The best min child weight is: 2
The best gamma is: 0.0757370018905035


### Step 5: Tune Stochastic Components

Now, we are finally ready to tune the stochastic components of the XGBoost model.  These parameters help prevent overfitting by reducing correlation between trees. Initialize your model with the best number of trees, best learning rate,and your optimized tree values (**Note**: you can use \**best_tree_parameters to unpack the the dictionary of optimzed tree values) .Then, define a parameter dictionary that takes on the following values:  

- `subsample` (Controls the fraction of training samples used for each boosting round) : A uniform distribution between .5 and .10 (remeber to check `scipy.stats.uniform()` documentation! )
- `colsample_bytree`(Specifies the fraction of features to consider when building each tree) : A uniform distribution between .5 and .10
- `random_state = 808`

We again use `RandomizedSearchCV` to find the best combination of these parameters. After fitting the model, print the best parameters. 


In [10]:
# Define subsequent model
model_3 = xgb.XGBClassifier(n_estimators = 29, 
                            learning_rate = random_search.best_params_["learning_rate"], 
                            max_depth = random_search_2.best_params_["max_depth"],
                            min_child_weight = random_search_2.best_params_["min_child_weight"],
                            gamma = random_search_2.best_params_["gamma"],
                            random_state = 808,
                           n_jobs = -1)

# Define stochastic parameters
param_dist = {
    "subsample": uniform(0.5, 0.5),
    "colsample_bytree": uniform(0.5, 0.5)
}

# Set up RandomizedSearchCV
random_search_3 = RandomizedSearchCV(
    model_3, 
    param_distributions = param_dist, 
    n_iter = 20, 
    verbose = 1, 
    random_state = 808)

# Run random search
random_search_3.fit(X_train, y_train, verbose = 0)

print(f'The best subsample is: {random_search_3.best_params_["subsample"]}')
print(f'The best colsample by tree is: {random_search_3.best_params_["colsample_bytree"]}')

Fitting 5 folds for each of 20 candidates, totalling 100 fits
The best subsample is: 0.940448212891594
The best colsample by tree is: 0.9874386808985325


### Step 6: Final Model Training and Evaluation

With the best hyperparameters selected, we now train the final model on the full training dataset and evaluate it on the separate evaluation dataset.

1. Prepare the evaluation data in the same manner as you did the training data

2. Train final model using the best parameters found in previous tuning steps (`best_tree_params`, `best_stochastic_params`).Set  `eval_metric = "logloss"` 

3. Fit the model to the full training dataset and predict on the evaluation data 


In [11]:
# Predict on eval data
X_eval = eval_data.drop('Angaus_obs', axis = 1).copy()
y_eval = eval_data['Angaus_obs']

# prepare eval data
le = LabelEncoder()
categorical = X_eval.select_dtypes(include = ['object'])

for column in categorical:
    X_eval[column] = le.fit_transform(X_eval[column])

# Check
X_eval.head()

,SegSumT,SegTSeas,SegLowFlow,DSDist,DSMaxSlope,USAvgT,USRainDays,USSlope,USNative,DSDam,Method,LocSed
0,16.6,1.01,1.017,5.23,0.29,-1.40,1.980,10.0,1.00,0,0,4.9
1,16.8,-0.51,1.002,2.24,0.00,0.27,0.460,0.7,0.00,0,0,2.3
2,16.3,0.76,1.023,162.28,5.14,-0.60,0.806,21.4,0.66,0,0,4.3
3,15.6,1.56,1.003,4.05,0.57,1.14,3.300,0.9,0.75,0,2,1.0
4,14.6,-0.20,1.023,127.03,1.72,-1.90,1.940,28.9,0.97,0,0,NaN


In [12]:
# Define final model
model_final = xgb.XGBClassifier(n_estimators = 29, 
                                learning_rate = random_search.best_params_["learning_rate"], 
                                max_depth = random_search_2.best_params_["max_depth"],
                                min_child_weight = random_search_2.best_params_["min_child_weight"],
                                gamma = random_search_2.best_params_["gamma"],
                                subsample = random_search_3.best_params_["subsample"],
                                colsample_bytree = random_search_3.best_params_["colsample_bytree"],
                                eval_metric = "logloss",
                                random_state = 808,
                               n_jobs = -1)
# Fit final model
model_final.fit(X, y, verbose = 0)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.9874386808985325, device=None,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='logloss', feature_types=None,
              gamma=0.0757370018905035, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.04204913369557677,
              max_bin=None, max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=9, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=29, n_jobs=-1,
              num_parallel_tree=None, random_state=808, ...)

### Step 7: Model Performance

Compute and print the AUC and feature importances for your model.

In [13]:
# Compute AUC
y_probs = model_final.predict_proba(X_eval)[:, 1]
roc_auc = roc_auc_score(y_eval, y_probs)

print(f"AUC Score: {roc_auc:.3f}")

# Extract feature importance
feature_importance = model_final.feature_importances_

# Create a DataFrame for better visualization
feature_names = X_train.columns  # Assuming X_train is a Pandas DataFrame
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importance})

# Sort by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

# Display the feature importance
print(importance_df)

AUC Score: 0.872
       Feature  Importance
0      SegSumT    0.288034
10      Method    0.142183
4   DSMaxSlope    0.088396
8     USNative    0.087599
6   USRainDays    0.070524
5       USAvgT    0.062914
7      USSlope    0.062694
11      LocSed    0.051671
3       DSDist    0.040877
1     SegTSeas    0.040633
2   SegLowFlow    0.033220
9        DSDam    0.031255


### Step 8: The comparison
How does your model's performance compare to the of Elith et al. (See Tables 2 and 3)?  Is there another way to compare the models in addition to predictive performance?  Whose model wins in that regard?

My model has a slightly larger AUC score (0.872) as compared to Elith et al (0.869). Based on the number of trees (29 vs. 1050, respectively), I would say that my model is more efficient computationally. We also have the same top feature `SegSumT`, but the features diverage after that. 